In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator
from quspin.tools.evolution import ED_state_vs_time, expm_multiply_parallel

plt.rcParams.update({
    "text.usetex": True,
})
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.fallback'] = 'stix'
plt.rcParams['font.family'] ='Times New Roman'
plt.style.use('seaborn-v0_8-deep')
prop_cycle = plt.rcParams['axes.prop_cycle']
dcolors = prop_cycle.by_key()['color']

# Model

The O'brian-Fendley model is much like the normal Majorana-Hubbard chain but with a slightly modified interaction term:

\begin{align*}
H_{OF} =& -it\sum_j \gamma_j \gamma_{j+1} -g \sum_j \gamma_{j-2}\gamma_{j-1}\gamma_{j+1}\gamma_{j+2}\\
=& t \sum_j \left(\sigma_j^x +\sigma_j^z \sigma_{j+1}^z\right)
+g \sum_j\left(\sigma_j^x \sigma_{j+1}^x \sigma_{j+2}^z + \sigma_j^z \sigma_{j+1}^z \sigma_{j+2}^z\right)
\end{align*}

In [ ]:
def Hof(t, g, basis):
    """
    t, g are couplings
    basis is quspin spinless_fermion_basis_1d
    """
    
    L = basis.N
    
    x_lst = [[-2*t, i] for i in range(L)]
    
    zz_lst = [[-2*t, i, i+1] for i in range(L-1)]
    
    xzz_lst = [[g, i, i+1, i+2] for i in range(L-2)]
    zzx_lst = [[g, i, i+1, i+2] for i in range(L-2)]

    zz_lst += [[-2*t, L-1, 0]]
    
    xzz_lst += [[g, L-2, L-1, 0],
                [g, L-1, 0, 1]]
    
    zzx_lst += [[g, L-2, L-1, 0],
                [g, L-1, 0, 1]]
    
    H = quantum_operator({'static': [['x', x_lst],
                                     ['zz', zz_lst],
                                     ['xzz', xzz_lst],
                                     ['zzx', zzx_lst],
                                    ]}, 
                         basis=basis, check_symm=False, check_herm=False)
    return H

# Time evolution

Here, I prepare the ground state of the Hamiltonian with $g/t=-1$. At time $t=0$, I turn off the interactions ($g=0$) and let the state evolve. Here, I do this in three ways: first, I explicitly form the propagator $U = e^{-i H t}$ and multiply the $t=0$ state. I also use QuSpin's built-in time-evolution code (which solves the time-dependent Schrodinger equation directly and can handle time-dependent Hamiltonians). Finally, I compare both of these to results obtained by forming the propagator $U$ using sparse matrix exponentiation.

In [ ]:
L = 8
basis = spin_basis_1d(L, pauli=1) # pauli matrix, not spin 1/2 (matches iTensor "S=1/2")

In [ ]:
H0 = Hof(1, -1, basis)
Ht = Hof(1, 0, basis)
steps = 51
times = np.linspace(0, 1, steps)

e, v = H0.eigh()
v0 = v[:,0] # grabbing just the ground state
et, vt = Ht.eigh()

energies = np.zeros((steps, 3))
olaps = np.zeros((steps, 3), dtype=np.complex128)

# Simple exponentiation code
for i, t in enumerate(times):
    U = vt @ np.diag(np.exp(-1j*et*t)) @ vt.T # propagator U = e^{-i H t}
    vi = U.dot(v0) # evolving t=0 state up to time t
    vi *= 1./np.linalg.norm(vi)
    olaps[i, 0] = np.vdot(v0, vi)
    energies[i, 0] = Ht.expt_value(vi).real

# Solving Schrodinger equation
vs = ED_state_vs_time(v0, et, vt, times, iterate=True)
for i, vi in enumerate(vs):
    vi *= 1./np.linalg.norm(vi)
    olaps[i,1] = np.sqrt(np.vdot(v0, vi)*np.vdot(vi, v0)) # being very explicit, just in case
    energies[i,1] = Ht.expt_value(vi).real

# Sparse exponentiation
Htcsr = Ht.tocsr() # getting Hamiltonian in Compressed Sparse Row format
U = expm_multiply_parallel(Htcsr, a=-1j*(times[1]-times[0])) # Propagator for a single timestep
vi = v0.copy()
for i, t in enumerate(times):
    # First, compute overlaps and energies
    olaps[i,2] = np.vdot(v0, vi)
    energies[i,2] = Ht.expt_value(vi).real
    # Then, evolve state one more time step by multiplying times U
    vi = U.dot(vi)
    vi *= 1./np.linalg.norm(vi) # normalization is Still Important!

In [ ]:
plt.figure(figsize=(6,6), dpi=200)
plt.subplot(2,1,1)
for i in range(3):
    plt.scatter(times, np.abs(olaps[:,i]), marker=str(i+1), label=f'Method {i+1}')
plt.xlabel(r'$t$')
plt.ylabel(r'$\langle \psi(0)|\psi(t)\rangle$')
plt.legend()

plt.subplot(2,1,2)
for i in range(3):
    plt.scatter(times, energies[:,i], marker=str(i+1), label=f'Method {i+1}')
plt.xlabel(r'$t$')
plt.ylabel(r'$\langle \psi(t)|H(t)|\psi(t)\rangle$')

plt.tight_layout()

It looks like these methods agree pretty well.